In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [12]:
bucket='sagemaker-builtin-imageclass-cancer-detection'
input_prefix='breast-cancer-detection/input/recordio'
input_train='s3://{}/{}/train'.format(bucket,input_prefix)
input_test='s3://{}/{}/test'.format(bucket,input_prefix)

In [3]:
output_prefix='breast-cancer-detection/output'
output_path='s3://{}/{}/'.format(bucket,output_prefix)

In [19]:
# Cofiguring the hyperparameters

num_classes=2
num_trainnig_samples=194266
num_layers=18
mini_batch_size=128
image_shape='3,50,50'
augmentation_type='crop_color_transform'
epochs=5
learning_rate=0.01
use_pretrained_model=1



In [20]:
instance_count=1
instance_type='ml.p2.16xlarge'
volume_size_gb=50

In [21]:
role=get_execution_role()
training_image=get_image_uri(boto3.Session().region_name,'image-classification')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [22]:
train_timeout=360000


In [27]:
session=sagemaker.Session()


In [28]:
estimator=sagemaker.estimator.Estimator(training_image,
                                       role,
                                       train_instance_count=instance_count,
                                       train_instance_type=instance_type,
                                       train_volume_size=volume_size_gb,
                                       train_max_run=train_timeout,
                                       output_path=output_path,
                                       base_job_name='sm-builtin-img-classification',
                                       sagemaker_session=session
                                       )

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [29]:
estimator.set_hyperparameters(num_classes=num_classes,
                             num_training_samples=1,
                             num_layers=num_layers,
                             mini_batch_size=mini_batch_size,
                             image_shape=image_shape,
                             augmentation_type=augmentation_type,
                             epochs=epochs,
                             learning_rate=learning_rate,
                             use_pretrained_model=use_pretrained_model)

In [ ]:
estimator.fit(inputs=input_train)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-06 10:36:25 Starting - Starting the training job...
2020-10-06 10:36:28 Starting - Launching requested ML instances.......